### Github上で Google が管理しているリポジトリをスクレイピングし、以下の情報を取得する

・リポジトリ名

・主要な言語

・スターの数

In [64]:
import sqlite3
import time
import requests
from bs4 import BeautifulSoup

In [65]:
# --- データベース関連の基本設定 ---
# DBファイルの保存先パス
path = ''
# DBファイル名
db_name = 'kadai.db'

# --- データベースとテーブルの準備 ---
print('データベースの準備を開始します...')
try:
    # DB接続オブジェクトの作成
    # データベースファイルが存在しない場合は新規作成される
    conn = sqlite3.connect(path + db_name)
    # SQLを実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

     # 既存のテーブルがあれば削除する
    cur.execute('DROP TABLE IF EXISTS repositories;')
    
    # SQL文の作成
    # repositoriesテーブルがなければ作成する (IF NOT EXISTS)
    #   - id: 主キー、自動で番号が振られる
    #   - name: リポジトリ名 (TEXT型、NULLを許可しない)
    #   - language: 主要言語 (TEXT型)
    #   - stars: スター数 (INTEGER型、NULLを許可しない)
    sql = '''
    CREATE TABLE IF NOT EXISTS repositories (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        language TEXT,
        stars INTEGER NOT NULL
    );
    '''
    
    # SQL文の実行
    cur.execute(sql)

except sqlite3.Error as e:
    print(f'エラーが発生しました: {e}')

finally:
    # DBへの接続を閉じる
    # tryブロックでエラーが発生しても、必ず接続は閉じられる
    if 'conn' in locals() and conn:
        conn.close()
    print('データベースの準備が完了しました。')

データベースの準備を開始します...
データベースの準備が完了しました。


In [66]:
# --- GitHubからデータをスクレイピング ---
# スクレイピング対象のURL
target_url = 'https://github.com/2y2y-ui/dsprog2'
# 取得したデータを保存するためのリスト
repo_data_list = []
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

print('スクレイピングを開始します...')
try:
    # サーバーに負荷をかけないよう1秒待機する
    time.sleep(1)
    
    # Webページを取得する
    response = requests.get(target_url, headers=headers)
    # ★修正点1: ステータスコードのチェックを追加
    response.raise_for_status()

    # 取得したHTMLをBeautifulSoupで解析する
    soup = BeautifulSoup(response.text, 'html.parser')

    # ★修正点2: ループは不要なので削除。ページ全体から各要素を1回だけ取得する

    # --- リポジトリ名の取得 ---
    name_elem = soup.select_one('strong[itemprop="name"] a')
    # 要素が見つかった場合のみ、テキストを取得。見つからなければ処理を中断。
    if not name_elem:
        raise ValueError("リポジトリ名が見つかりませんでした。")
    name = name_elem.text.strip()
    
    # --- 主要言語の取得 ---
    # サイドバーの言語リストの一番上にあるものを取得する
    lang_elem = soup.select_one('div.Layout-sidebar ul li a span:first-child')
    language = lang_elem.text.strip() if lang_elem else 'N/A' # 言語がない場合もある

    # --- スター数の取得 ---
    # IDで指定された要素から取得するのが最も確実
    star_elem = soup.select_one('#repo-stars-counter-star')
    if star_elem:
        # title属性に正確な数値が入っているため、それを取得する
        stars_text = star_elem.get('title', '0').strip().replace(',', '')
        stars = int(stars_text)
    else:
        stars = 0
    
    # --- 取得したデータをタプルにまとめてリストに追加 ---
    repo_data_list.append((name, language, stars))
    print(f"取得成功: {name}")

except requests.exceptions.RequestException as e:
    print(f'リクエスト中にエラーが発生しました: {e}')
except ValueError as e:
    # 自分で発生させたエラーを捕捉する
    print(f'データ取得エラー: {e}')

finally:
    print(f'スクレイピングが完了しました。{len(repo_data_list)}件のデータを取得しました。')

スクレイピングを開始します...
取得成功: dsprog2
スクレイピングが完了しました。1件のデータを取得しました。


In [67]:
# --- 取得したデータをデータベースに保存 ---
if repo_data_list:
    print('データベースへのデータ保存を開始します...')
    try:
        conn = sqlite3.connect(path + db_name)
        cur = conn.cursor()

        sql = 'INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?);'
        cur.executemany(sql, repo_data_list)
        conn.commit()
    except sqlite3.Error as e:
        print(f'エラーが発生しました: {e}')
    finally:
        if 'conn' in locals() and conn:
            conn.close()
        print('データベースへのデータ保存が完了しました。')
else:
    print('保存するデータがありませんでした。')

データベースへのデータ保存を開始します...
データベースへのデータ保存が完了しました。


In [68]:
# --- データベース関連の基本設定 ---
# --- データベースの内容を確認 ---
print('データベースの内容を表示します...')
print('--------------------------')
try:
    # DB接続オブジェクトの作成
    conn = sqlite3.connect(path + db_name)
    # SQLを実行するためのカーソルオブジェクトを取得
    cur = conn.cursor()

    
    # データを参照するSQL
    sql = 'SELECT id, name, language, stars FROM repositories;'
    
    # SQL文の実行と結果の表示
    # 実行結果はイテラブル（繰り返し可能）なので、forループで1行ずつ取り出せる
    for row in cur.execute(sql):
        # rowはタプル形式 (0: id, 1: name, 2: language, 3: stars)
        print(f"ID: {row[0]}, Name: {row[1]}, Language: {row[2]}, Stars: {row[3]}")

except sqlite3.Error as e:
    print(f'エラーが発生しました: {e}')

finally:
    # DBへの接続を閉じる
    if 'conn' in locals() and conn:
        conn.close()
    print('--------------------------')
    print('データ表示が完了しました。')

データベースの内容を表示します...
--------------------------
ID: 1, Name: dsprog2, Language: N/A, Stars: 0
--------------------------
データ表示が完了しました。
